# Percolación Ejercicio 1
***
## Grupo:
1. Adrián Bedón
1. Merlo José Miguel
1. Moreta Andrés
1. Ocaña Dennis
1. Ramos Xavier
***

## Qué es la percolación?
Percolación hace referencia al paso de fluidos a travez de materiales porosos. Un ejemplo de percolación es la filtracion<br>
<br>
<div>
<img src="https://cdn0.ecologiaverde.com/es/posts/3/2/1/como_hacer_un_filtro_de_agua_casero_para_beber_1123_orig.jpg" width="500"/>
</div>

***

## Simulación de Montecarlo para estudiar un fenómeno natural conocido como percolación

Para el proposito de estas simulaciones se definirá un sistema de `n * n` a modo de cuaadrícula. Dentro de esta cada cuadrante puede estar bloqueado o abierto y los cuadrantes abiertos son inicializados _vacíos_. Un sitio _lleno_ es un sitio abierto que se conecta a un sitio aledaño que también está _abierto_ siendo estos: arriba, abajo, izquierda y/o derecha).<br>
<br>
El sistema se dirá que percola si es que hay un sitio abierto _completo_ esto significando que esta conectado de un extremo a otro de la cuadrícula.<br>
<br>
<div>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-yes.png" width="300"/>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-no.png" width="300"/>
</div>

***